In [ ]:
#Get the packages
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
#Get the required attributes from the item table
df_itm_slice = df_itm[['TransactionID','ArticleID','ItemReceiptDescription', 'Quantity']].sort_values('TransactionID',ascending=True).dropna()
df_itm_slice.info()

In [ ]:
#Calculate how many of each item are in each transaction
df =df_itm_slice.groupby(['TransactionID', 'ItemReceiptDescription'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('TransactionID')
df.head()

In [ ]:
#Create the encoding function for the Apriori Algorithm
def encode_units(x):
    if x<=0:
        return 0
    if x>0:
        return 1

In [ ]:
#Encode the dataset
basket_encode = df.applymap(encode_units)
basket_encode.head()

In [ ]:
#Remove transactions that don't have at least two distinct items in them
basket_encode_2 = basket_encode[(basket_encode>0).sum(axis=1)>=2]
basket_encode_2.head()

In [ ]:
#Calulate the frequent items
freq_itms = apriori(basket_encode_2, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
freq_itms['length'] = freq_itms['itemsets'].apply(lambda x: len(x))
freq_itms

In [ ]:
#apply the association rules
associations = association_rules(freq_itms, metric='lift', min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)
associations.head(25)